In [1]:
from RSA import RSA
from OEAP import OEAP
from AES import AES
import base64
import hashlib

In [2]:
rsa = RSA(10)
oeap = OEAP(1024, 512)

# Parte I: Geração de chaves

In [3]:

keys = rsa.key_gen()
keys

{'private': (476789, 736139), 'public': (1565, 736139)}

O crivo de Erathostenes gera os primos entre 0 e 1024, após isso, é encontrado um número que não seja divisível por nenhum dos primos gerados.

Caso esse número passe no teste de Miller Robin, a probabilidade dele ser um primo é altíssima.

O algoritmo `key_gen` é responsável por gerar as chaves

```
    def key_gen(self, pq=None):
        if not pq:
            p = self.gen_prime()
            q = self.gen_prime()
        else:
            p, q = pq

        while p == q:
            q = self.gen_prime()
        
        n = p*q
        lambda_n = (p-1)*(q-1)
        e = random.randrange(1,lambda_n)

        while math.gcd(e, lambda_n) != 1:
            e = random.randrange(1,lambda_n)
        
        d = RSA.eucalg(e, lambda_n)[0]
        if d < 0: d += lambda_n

        return {"private": (d, n), "public": (e, n)}
```

# Parte II: Cifra Simétrica

In [4]:
key = AES.create_key() # geração da chave
enc_key = oeap.encrypt(key) # Cifração assimétrica da chave de sessão
len(key)

32

In [47]:
aes = AES(bytes.fromhex(key))


msg = "6bc1bee22e409f96e93d7e117393172a"

cip = aes.cipher(bytes.fromhex(msg))
cip.hex()

'92015edb78ba3ab51b1f7d4879787d52'

In [48]:
dec = aes.decipher(cip)
dec.hex()

'6bc1bee22e409f96e93d7e117393172a'

# Parte III: Geração de assinatura

In [53]:
sha3 = hashlib.sha3_256()
sha3.update("Opa meu rei, tudo certo?".encode())

msg_hash = sha3.digest()

signed = rsa.cipher(keys["public"], msg_hash.decode('latin-1'))
signed

[431238,
 0,
 454413,
 68095,
 593974,
 306262,
 434173,
 337689,
 561615,
 479858,
 249157,
 30304,
 331722,
 543786,
 477188,
 146903,
 217072,
 657536,
 59998,
 373977,
 593974,
 473322,
 377656,
 30304,
 190749,
 578878,
 407095,
 175840,
 354867,
 473322,
 191943,
 190749]

In [54]:
textt = rsa.decipher(keys["private"], signed)

if msg_hash == textt.encode('latin-1'):
    print("Valid")
else:
    print("Invalid")

Valid
